# 🌍 PASO 1: TRATAMIENTO DE DATOS - Sistema de Calidad del Aire

## 📋 Objetivo
Obtener, limpiar y preparar datos de calidad del aire de múltiples fuentes para análisis y modelado.

## 🎯 Fuentes de Datos
1. **🛰️ NASA TEMPO** - Satélite geoestacionario
2. **📡 OpenAQ** - Red global de estaciones terrestres
3. **🌤️ NASA POWER** - Datos meteorológicos

## 📊 Variables a Obtener
- **Contaminantes**: PM2.5, PM10, O₃, NO₂
- **Meteorología**: Temperatura, Humedad, Viento
- **Índice**: AQI (Air Quality Index)

---

## 1️⃣ CONFIGURACIÓN INICIAL

In [9]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Librerías importadas correctamente")
print(f"📅 Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Librerías importadas correctamente
📅 Fecha de ejecución: 2025-10-03 20:45:59


In [10]:
# Parámetros de configuración
UBICACION = "Los Angeles, CA"
LATITUD = 34.0522
LONGITUD = -118.2437

# Período de datos
FECHA_INICIO = "20240901"  # YYYYMMDD
FECHA_FIN = "20241001"     # YYYYMMDD

print("="*80)
print("🌍 CONFIGURACIÓN DEL PROYECTO")
print("="*80)
print(f"📍 Ubicación: {UBICACION}")
print(f"🌐 Coordenadas: ({LATITUD}°N, {LONGITUD}°W)")
print(f"📅 Período: {FECHA_INICIO} → {FECHA_FIN}")
print("="*80)

🌍 CONFIGURACIÓN DEL PROYECTO
📍 Ubicación: Los Angeles, CA
🌐 Coordenadas: (34.0522°N, -118.2437°W)
📅 Período: 20240901 → 20241001


---
## 2️⃣ OBTENCIÓN DE DATOS

### 📡 Estrategia de Obtención
1. Intentar obtener datos de **OpenAQ** (estaciones terrestres - mayor precisión local)
2. Complementar con **NASA TEMPO** (satélite - cobertura completa)
3. Añadir datos **meteorológicos** (NASA POWER)
4. **Fusionar** todas las fuentes

In [ ]:
# Verificar credenciales necesarias
import os

print("🔐 Verificando credenciales...\n")

# NASA Earthdata
NASA_USERNAME = os.getenv('NASA_USERNAME', '')
NASA_PASSWORD = os.getenv('NASA_PASSWORD', '')

if NASA_USERNAME and NASA_PASSWORD:
    print("✅ Credenciales NASA encontradas")
else:
    print("⚠️ Credenciales NASA no encontradas")
    print("   Configura las variables de entorno NASA_USERNAME y NASA_PASSWORD")

# OpenAQ API Key
OPENAQ_API_KEY = os.getenv('OPENAQ_API_KEY', '')

if OPENAQ_API_KEY:
    print("✅ API Key de OpenAQ encontrada")
else:
    print("⚠️ API Key de OpenAQ no encontrada")
    print("   Configura la variable de entorno OPENAQ_API_KEY")
    print("   Obtén tu key en: https://openaq.org/")

print("\n" + "="*80)

🔐 Verificando credenciales...

✅ Credenciales NASA encontradas
✅ API Key de OpenAQ encontrada



### 📌 NOTA IMPORTANTE

Antes de continuar, necesitas configurar tus credenciales:

**1. NASA Earthdata (para TEMPO)**
- Regístrate en: https://urs.earthdata.nasa.gov/
- Configura las variables de entorno:
  ```python
  # Opción 1: En el código (temporal)
  NASA_USERNAME = "tu_usuario"
  NASA_PASSWORD = "tu_contraseña"
  
  # Opción 2: Variables de entorno (recomendado)
  # En PowerShell:
  # $env:NASA_USERNAME = "tu_usuario"
  # $env:NASA_PASSWORD = "tu_contraseña"
  ```

**2. OpenAQ API (para estaciones terrestres)**
- Regístrate en: https://openaq.org/
- Obtén tu API key
- Configura:
  ```python
  OPENAQ_API_KEY = "tu_api_key"
  ```

**3. Ejecutar la siguiente celda para configurar manualmente (si es necesario)**

In [8]:
# ⚙️ CONFIGURACIÓN MANUAL (si las variables de entorno no están disponibles)
# Descomenta y completa con tus credenciales:

# NASA_USERNAME = "tu_usuario_nasa"
# NASA_PASSWORD = "tu_contraseña_nasa"
# OPENAQ_API_KEY = "tu_api_key_openaq"

print("⚙️ Configuración manual lista")
print("   Asegúrate de descomentar y completar las credenciales")

⚙️ Configuración manual lista
   Asegúrate de descomentar y completar las credenciales


---
## 3️⃣ PRÓXIMOS PASOS

En las siguientes secciones desarrollaremos:

1. **Cliente OpenAQ** - Obtención de datos de estaciones terrestres
2. **Cliente TEMPO** - Obtención de datos satelitales
3. **Integración Meteorológica** - NASA POWER
4. **Fusión de Datos** - Combinar todas las fuentes
5. **Limpieza de Datos** - Eliminar outliers, interpolar valores faltantes
6. **Análisis Exploratorio** - Visualizar y entender los datos
7. **Exportación** - Guardar datos limpios para modelado

---

### 🎯 ¿Listo para continuar?

**Ejecuta todas las celdas anteriores** y estarás listo para comenzar con la obtención de datos.

**Siguiente:** Implementaremos el cliente de OpenAQ paso a paso.

---
## 4️⃣ EXPLORACIÓN DE DATOS - OPENAQ (Estaciones Terrestres)

### 📡 ¿Qué es OpenAQ?
Red global de estaciones de monitoreo de calidad del aire que recopila datos de más de 10,000 estaciones en tiempo real.

**Ventajas:**
- ✅ Datos en tiempo real
- ✅ Precisión local alta
- ✅ Múltiples contaminantes
- ✅ API gratuita

**Variables disponibles:**
- PM2.5, PM10, O₃, NO₂, CO, SO₂

In [16]:
# Cliente básico para OpenAQ API v3
import requests
import json

class OpenAQExplorer:
    """Explorador simple de datos OpenAQ"""
    
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "https://api.openaq.org/v3"
        self.headers = {"X-API-Key": api_key}
    
    def buscar_estaciones_cercanas(self, latitud, longitud, radio_km=50, limit=10):
        """Busca estaciones cercanas a una ubicación"""
        
        print(f"🔍 Buscando estaciones cercanas a ({latitud}, {longitud})")
        print(f"📏 Radio de búsqueda: {radio_km} km")
        print("-" * 80)
        
        # Endpoint de locations con bounding box
        url = f"{self.base_url}/locations"
        
        # Calcular bounding box aproximado
        lat_offset = radio_km / 111  # 1 grado ≈ 111 km
        lon_offset = radio_km / (111 * np.cos(np.radians(latitud)))
        
        params = {
            "bbox": f"{longitud-lon_offset},{latitud-lat_offset},{longitud+lon_offset},{latitud+lat_offset}",
            "limit": limit,
            "order_by": "name"
        }
        
        try:
            response = requests.get(url, headers=self.headers, params=params)
            response.raise_for_status()
            data = response.json()
            
            if 'results' in data and len(data['results']) > 0:
                estaciones = data['results']
                print(f"✅ Encontradas {len(estaciones)} estaciones\n")
                
                for i, est in enumerate(estaciones, 1):
                    nombre = est.get('name', 'Sin nombre')
                    ciudad = est.get('locality', 'N/A')
                    pais = est.get('country', {}).get('name', 'N/A')
                    
                    # Parámetros disponibles
                    parametros = [p.get('displayName', '') 
                                 for p in est.get('parameters', [])]
                    
                    print(f"{i}. 📍 {nombre}")
                    print(f"   • Ubicación: {ciudad}, {pais}")
                    print(f"   • Parámetros: {', '.join(parametros[:5])}")  # Primeros 5
                    print(f"   • ID: {est.get('id')}")
                    print()
                
                return estaciones
            else:
                print("⚠️ No se encontraron estaciones en el área")
                return []
                
        except requests.exceptions.RequestException as e:
            print(f"❌ Error al buscar estaciones: {e}")
            print(f"   Respuesta: {response.text[:500] if 'response' in locals() else 'N/A'}")
            return []
    
    def obtener_mediciones_estacion(self, location_id, fecha_inicio, fecha_fin, parametro="pm25"):
        """Obtiene mediciones de una estación específica"""
        
        print(f"📊 Obteniendo mediciones de estación ID: {location_id}")
        print(f"📅 Período: {fecha_inicio} → {fecha_fin}")
        print(f"🔬 Parámetro: {parametro}")
        print("-" * 80)
        
        url = f"{self.base_url}/measurements"
        params = {
            "locations_id": location_id,
            "datetime_from": f"{fecha_inicio}T00:00:00Z",
            "datetime_to": f"{fecha_fin}T23:59:59Z",
            "parameters_id": self._get_parameter_id(parametro),
            "limit": 1000
        }
        
        try:
            response = requests.get(url, headers=self.headers, params=params)
            response.raise_for_status()
            data = response.json()
            
            if 'results' in data:
                mediciones = data['results']
                print(f"✅ Obtenidas {len(mediciones)} mediciones\n")
                
                if len(mediciones) > 0:
                    # Crear DataFrame
                    df = pd.DataFrame([
                        {
                            'timestamp': m['datetime'],
                            'value': m['value'],
                            'unit': m['parameter']['units'],
                            'parameter': m['parameter']['displayName']
                        }
                        for m in mediciones
                    ])
                    
                    df['timestamp'] = pd.to_datetime(df['timestamp'])
                    df = df.sort_values('timestamp')
                    
                    # Mostrar resumen
                    print("📋 RESUMEN DE DATOS:")
                    print(f"   • Registros: {len(df)}")
                    print(f"   • Período: {df['timestamp'].min()} a {df['timestamp'].max()}")
                    print(f"   • Valor promedio: {df['value'].mean():.2f} {df['unit'].iloc[0]}")
                    print(f"   • Valor mínimo: {df['value'].min():.2f}")
                    print(f"   • Valor máximo: {df['value'].max():.2f}")
                    
                    return df
                else:
                    print("⚠️ No hay mediciones en el período especificado")
                    return pd.DataFrame()
                    
        except requests.exceptions.RequestException as e:
            print(f"❌ Error al obtener mediciones: {e}")
            print(f"   Respuesta: {response.text[:500] if 'response' in locals() else 'N/A'}")
            return pd.DataFrame()
    
    def _get_parameter_id(self, parametro):
        """Mapea nombres comunes a IDs de parámetros OpenAQ"""
        mapping = {
            'pm25': 2,
            'pm10': 1,
            'o3': 3,
            'no2': 4,
            'co': 5,
            'so2': 6
        }
        return mapping.get(parametro.lower(), 2)

# Crear explorador
explorer_openaq = OpenAQExplorer(OPENAQ_API_KEY)
print("✅ Explorador OpenAQ creado")

✅ Explorador OpenAQ creado


In [17]:
# Buscar estaciones cercanas a nuestra ubicación
estaciones = explorer_openaq.buscar_estaciones_cercanas(
    LATITUD, 
    LONGITUD, 
    radio_km=50,
    limit=10
)

🔍 Buscando estaciones cercanas a (34.0522, -118.2437)
📏 Radio de búsqueda: 50 km
--------------------------------------------------------------------------------
✅ Encontradas 10 estaciones

1. 📍 South Long Beach
   • Ubicación: None, United States
   • Parámetros: 
   • ID: 847

2. 📍 Pasadena
   • Ubicación: None, United States
   • Parámetros: 
   • ID: 1019

3. 📍 Pico Rivera
   • Ubicación: None, United States
   • Parámetros: 
   • ID: 1036

4. 📍 Pomona
   • Ubicación: Los Angeles-Long Beach-Santa Ana, United States
   • Parámetros: 
   • ID: 1052

5. 📍 Glendora - Laurel
   • Ubicación: Los Angeles-Long Beach-Santa Ana, United States
   • Parámetros: 
   • ID: 1200

6. 📍 LAX-Hastings
   • Ubicación: None, United States
   • Parámetros: 
   • ID: 1247

7. 📍 La Habra
   • Ubicación: None, United States
   • Parámetros: 
   • ID: 1268

8. 📍 Azusa
   • Ubicación: None, United States
   • Parámetros: 
   • ID: 1326

9. 📍 West Los Angeles - V
   • Ubicación: None, United States
   • Pará

In [18]:
# Obtener mediciones de la estación más cercana (si hay)
if len(estaciones) > 0:
    estacion_id = estaciones[0]['id']
    
    # Convertir fechas al formato correcto (YYYY-MM-DD)
    fecha_inicio_openaq = f"{FECHA_INICIO[:4]}-{FECHA_INICIO[4:6]}-{FECHA_INICIO[6:]}"
    fecha_fin_openaq = f"{FECHA_FIN[:4]}-{FECHA_FIN[4:6]}-{FECHA_FIN[6:]}"
    
    # Obtener datos de PM2.5
    df_openaq_pm25 = explorer_openaq.obtener_mediciones_estacion(
        estacion_id,
        fecha_inicio_openaq,
        fecha_fin_openaq,
        parametro="pm25"
    )
    
    if not df_openaq_pm25.empty:
        print("\n" + "="*80)
        print("📊 MUESTRA DE DATOS (primeros 10 registros):")
        print("="*80)
        print(df_openaq_pm25.head(10))
else:
    print("⚠️ No se encontraron estaciones para explorar")
    df_openaq_pm25 = pd.DataFrame()

📊 Obteniendo mediciones de estación ID: 847
📅 Período: 2024-09-01 → 2024-10-01
🔬 Parámetro: pm25
--------------------------------------------------------------------------------
❌ Error al obtener mediciones: 404 Client Error: Not Found for url: https://api.openaq.org/v3/measurements?locations_id=847&datetime_from=2024-09-01T00%3A00%3A00Z&datetime_to=2024-10-01T23%3A59%3A59Z&parameters_id=2&limit=1000
   Respuesta: {"detail":"Not Found"}
❌ Error al obtener mediciones: 404 Client Error: Not Found for url: https://api.openaq.org/v3/measurements?locations_id=847&datetime_from=2024-09-01T00%3A00%3A00Z&datetime_to=2024-10-01T23%3A59%3A59Z&parameters_id=2&limit=1000
   Respuesta: {"detail":"Not Found"}


In [ ]:
# Visualizar datos de OpenAQ
if not df_openaq_pm25.empty:
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    fig.suptitle(f'📡 Datos OpenAQ - PM2.5 ({UBICACION})', fontsize=16, fontweight='bold')
    
    # 1. Serie temporal completa
    axes[0, 0].plot(df_openaq_pm25['timestamp'], df_openaq_pm25['value'], 
                    linewidth=1.5, color='steelblue', alpha=0.7)
    axes[0, 0].fill_between(df_openaq_pm25['timestamp'], df_openaq_pm25['value'], 
                            alpha=0.3, color='lightblue')
    axes[0, 0].set_xlabel('Fecha', fontweight='bold')
    axes[0, 0].set_ylabel(f"PM2.5 ({df_openaq_pm25['unit'].iloc[0]})", fontweight='bold')
    axes[0, 0].set_title('1️⃣ Serie Temporal Completa', fontweight='bold')
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # 2. Distribución (histograma)
    axes[0, 1].hist(df_openaq_pm25['value'], bins=50, color='coral', 
                    alpha=0.7, edgecolor='black')
    axes[0, 1].axvline(df_openaq_pm25['value'].mean(), color='red', 
                       linestyle='--', linewidth=2, label=f"Media: {df_openaq_pm25['value'].mean():.2f}")
    axes[0, 1].axvline(df_openaq_pm25['value'].median(), color='green', 
                       linestyle='--', linewidth=2, label=f"Mediana: {df_openaq_pm25['value'].median():.2f}")
    axes[0, 1].set_xlabel(f"PM2.5 ({df_openaq_pm25['unit'].iloc[0]})", fontweight='bold')
    axes[0, 1].set_ylabel('Frecuencia', fontweight='bold')
    axes[0, 1].set_title('2️⃣ Distribución de Valores', fontweight='bold')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3, axis='y')
    
    # 3. Box plot
    axes[1, 0].boxplot(df_openaq_pm25['value'], vert=True, patch_artist=True,
                       boxprops=dict(facecolor='lightgreen', alpha=0.7),
                       medianprops=dict(color='red', linewidth=2),
                       whiskerprops=dict(linewidth=1.5),
                       capprops=dict(linewidth=1.5))
    axes[1, 0].set_ylabel(f"PM2.5 ({df_openaq_pm25['unit'].iloc[0]})", fontweight='bold')
    axes[1, 0].set_title('3️⃣ Box Plot - Detección de Outliers', fontweight='bold')
    axes[1, 0].grid(True, alpha=0.3, axis='y')
    
    # Añadir estadísticas al box plot
    q1 = df_openaq_pm25['value'].quantile(0.25)
    q3 = df_openaq_pm25['value'].quantile(0.75)
    iqr = q3 - q1
    axes[1, 0].text(1.3, df_openaq_pm25['value'].max(), 
                    f"Q1: {q1:.2f}\nQ3: {q3:.2f}\nIQR: {iqr:.2f}", 
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # 4. Estadísticas por hora del día
    df_openaq_pm25['hour'] = df_openaq_pm25['timestamp'].dt.hour
    hourly_stats = df_openaq_pm25.groupby('hour')['value'].agg(['mean', 'std']).reset_index()
    
    axes[1, 1].plot(hourly_stats['hour'], hourly_stats['mean'], 
                    marker='o', linewidth=2, markersize=8, color='purple', label='Media')
    axes[1, 1].fill_between(hourly_stats['hour'], 
                            hourly_stats['mean'] - hourly_stats['std'],
                            hourly_stats['mean'] + hourly_stats['std'],
                            alpha=0.3, color='purple', label='±1 σ')
    axes[1, 1].set_xlabel('Hora del Día', fontweight='bold')
    axes[1, 1].set_ylabel(f"PM2.5 ({df_openaq_pm25['unit'].iloc[0]})", fontweight='bold')
    axes[1, 1].set_title('4️⃣ Patrón Horario', fontweight='bold')
    axes[1, 1].set_xticks(range(0, 24, 3))
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n✅ Visualización de datos OpenAQ completada")
else:
    print("⚠️ No hay datos para visualizar")

---
## 5️⃣ EXPLORACIÓN DE DATOS - NASA TEMPO (Satélite)

### 🛰️ ¿Qué es TEMPO?
Satélite geoestacionario de NASA que monitorea la calidad del aire sobre Norteamérica desde el espacio.

**Ventajas:**
- ✅ Cobertura espacial completa
- ✅ Resolución temporal horaria
- ✅ Datos NO₂ de alta calidad
- ✅ Sin gaps geográficos

**Variables principales:**
- NO₂ (Dióxido de Nitrógeno)
- Datos de nubes
- Calidad del aire estimada

In [ ]:
# Cliente básico para NASA TEMPO
import earthaccess

class TEMPOExplorer:
    """Explorador simple de datos TEMPO"""
    
    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.auth = None
        
    def autenticar(self):
        """Autenticar con NASA Earthdata"""
        print("🔐 Autenticando con NASA Earthdata...")
        try:
            self.auth = earthaccess.login(
                username=self.username,
                password=self.password
            )
            print("✅ Autenticación exitosa")
            return True
        except Exception as e:
            print(f"❌ Error de autenticación: {e}")
            return False
    
    def buscar_granulos(self, latitud, longitud, fecha_inicio, fecha_fin, limit=10):
        """Busca gránulos TEMPO para una ubicación y período"""
        
        print(f"\n🛰️ Buscando datos TEMPO")
        print(f"📍 Ubicación: ({latitud}, {longitud})")
        print(f"📅 Período: {fecha_inicio} → {fecha_fin}")
        print("-" * 80)
        
        try:
            # Buscar gránulos
            granulos = earthaccess.search_data(
                short_name="TEMPO_NO2_L3",
                cloud_hosted=True,
                bounding_box=(longitud-1, latitud-1, longitud+1, latitud+1),
                temporal=(fecha_inicio, fecha_fin),
                count=limit
            )
            
            if len(granulos) > 0:
                print(f"✅ Encontrados {len(granulos)} gránulos TEMPO\n")
                
                for i, granulo in enumerate(granulos[:5], 1):  # Mostrar solo primeros 5
                    print(f"{i}. 📦 {granulo['umm']['GranuleUR']}")
                    print(f"   • Fecha: {granulo['umm']['TemporalExtent']}")
                    print(f"   • Tamaño: {granulo['umm'].get('DataGranule', {}).get('ArchiveAndDistributionInformation', [{}])[0].get('Size', 'N/A')} MB")
                    print()
                
                return granulos
            else:
                print("⚠️ No se encontraron gránulos TEMPO para el período")
                return []
                
        except Exception as e:
            print(f"❌ Error al buscar gránulos: {e}")
            return []
    
    def descargar_muestra(self, granulos, num_archivos=3, directorio="tempo_sample"):
        """Descarga una muestra de archivos TEMPO"""
        
        import os
        
        if not os.path.exists(directorio):
            os.makedirs(directorio)
        
        print(f"⬇️ Descargando {num_archivos} archivos de muestra...")
        print(f"📁 Directorio: {directorio}")
        print("-" * 80)
        
        try:
            archivos_descargados = earthaccess.download(
                granulos[:num_archivos],
                directorio
            )
            
            print(f"\n✅ Descargados {len(archivos_descargados)} archivos:")
            for i, archivo in enumerate(archivos_descargados, 1):
                print(f"   {i}. {os.path.basename(archivo)}")
            
            return archivos_descargados
            
        except Exception as e:
            print(f"❌ Error al descargar: {e}")
            return []
    
    def leer_archivo_tempo(self, archivo_path):
        """Lee un archivo NetCDF de TEMPO"""
        
        import netCDF4 as nc
        
        print(f"\n📖 Leyendo archivo TEMPO...")
        print(f"📄 {os.path.basename(archivo_path)}")
        print("-" * 80)
        
        try:
            dataset = nc.Dataset(archivo_path, 'r')
            
            print("\n📊 INFORMACIÓN DEL ARCHIVO:")
            print(f"   • Variables: {len(dataset.variables)}")
            print(f"   • Dimensiones: {len(dataset.dimensions)}")
            
            print("\n🔬 VARIABLES PRINCIPALES:")
            for var_name in list(dataset.variables.keys())[:10]:  # Primeras 10
                var = dataset.variables[var_name]
                print(f"   • {var_name}: {var.shape} - {getattr(var, 'long_name', 'N/A')}")
            
            # Extraer NO2 si está disponible
            if 'vertical_column_troposphere' in dataset.variables:
                no2_data = dataset.variables['vertical_column_troposphere'][:]
                print(f"\n✅ Datos NO₂ encontrados: {no2_data.shape}")
                
                # Estadísticas básicas
                no2_valid = no2_data[~no2_data.mask] if hasattr(no2_data, 'mask') else no2_data
                print(f"   • Valores válidos: {len(no2_valid)}")
                print(f"   • Promedio: {no2_valid.mean():.2e}")
                print(f"   • Min: {no2_valid.min():.2e}")
                print(f"   • Max: {no2_valid.max():.2e}")
                
                dataset.close()
                return no2_data
            else:
                print("\n⚠️ Variable NO₂ no encontrada en este archivo")
                dataset.close()
                return None
                
        except Exception as e:
            print(f"❌ Error al leer archivo: {e}")
            return None

# Crear explorador TEMPO
explorer_tempo = TEMPOExplorer(NASA_USERNAME, NASA_PASSWORD)
print("✅ Explorador TEMPO creado")

In [ ]:
# Autenticar y buscar datos TEMPO
if explorer_tempo.autenticar():
    # Convertir fechas al formato YYYY-MM-DD
    fecha_inicio_tempo = f"{FECHA_INICIO[:4]}-{FECHA_INICIO[4:6]}-{FECHA_INICIO[6:]}"
    fecha_fin_tempo = f"{FECHA_FIN[:4]}-{FECHA_FIN[4:6]}-{FECHA_FIN[6:]}"
    
    granulos_tempo = explorer_tempo.buscar_granulos(
        LATITUD,
        LONGITUD,
        fecha_inicio_tempo,
        fecha_fin_tempo,
        limit=15
    )
else:
    print("⚠️ No se pudo autenticar con NASA. Verifica tus credenciales.")
    granulos_tempo = []

In [ ]:
# Descargar y explorar una muestra de archivos TEMPO
if len(granulos_tempo) > 0:
    archivos_tempo = explorer_tempo.descargar_muestra(
        granulos_tempo,
        num_archivos=3,
        directorio="tempo_sample"
    )
    
    # Leer el primer archivo como muestra
    if len(archivos_tempo) > 0:
        datos_no2 = explorer_tempo.leer_archivo_tempo(archivos_tempo[0])
else:
    print("⚠️ No hay gránulos TEMPO para descargar")
    archivos_tempo = []

---
## 6️⃣ COMPARACIÓN VISUAL: OpenAQ vs TEMPO

Ahora que tenemos datos de ambas fuentes, podemos compararlas:

In [ ]:
# Resumen comparativo de las fuentes de datos
print("="*80)
print("📊 COMPARACIÓN: OpenAQ vs TEMPO")
print("="*80)

# Tabla comparativa
comparacion = {
    'Característica': [
        'Tipo de fuente',
        'Cobertura espacial',
        'Resolución temporal',
        'Variables principales',
        'Precisión local',
        'Acceso a datos',
        'Latencia',
        'Gaps geográficos'
    ],
    'OpenAQ': [
        'Estaciones terrestres',
        'Puntual (ubicaciones fijas)',
        'Minutos/Horaria',
        'PM2.5, PM10, O₃, NO₂, CO, SO₂',
        'Alta (medición directa)',
        'API gratuita',
        'Tiempo real',
        'Sí (solo donde hay estaciones)'
    ],
    'TEMPO': [
        'Satélite geoestacionario',
        'Continental (Norteamérica)',
        'Horaria',
        'NO₂, nubes, AQI estimado',
        'Media (estimación satelital)',
        'NASA Earthdata (registro gratuito)',
        '1-2 horas',
        'No (cobertura completa)'
    ]
}

df_comparacion = pd.DataFrame(comparacion)
print(df_comparacion.to_string(index=False))

print("\n" + "="*80)
print("💡 CONCLUSIÓN:")
print("="*80)
print("""
✅ OpenAQ es ideal para:
   • Monitoreo en ubicaciones específicas
   • Datos en tiempo real
   • Múltiples contaminantes
   • Validación de modelos

✅ TEMPO es ideal para:
   • Cobertura espacial completa
   • Áreas sin estaciones terrestres
   • Análisis regional
   • Datos NO₂ de alta calidad

🎯 ESTRATEGIA ÓPTIMA:
   Usar AMBAS fuentes de forma complementaria:
   • OpenAQ para precisión local
   • TEMPO para cobertura espacial completa
   • Fusionar datos para mejor resultado
""")

---
## 7️⃣ PRÓXIMOS PASOS

### ✅ Lo que hemos logrado:
1. ✅ Configuración del proyecto
2. ✅ Exploración de datos OpenAQ (estaciones terrestres)
3. ✅ Exploración de datos TEMPO (satélite)
4. ✅ Comparación de ambas fuentes

### 🎯 Siguientes fases:

1. **Integración de datos meteorológicos** (NASA POWER)
   - Temperatura, humedad, viento
   - Complementar datos de calidad del aire

2. **Fusión de fuentes**
   - Combinar OpenAQ + TEMPO + Meteorología
   - Estrategia de fusión ponderada

3. **Limpieza de datos**
   - Eliminar duplicados
   - Interpolar valores faltantes
   - Detectar y eliminar outliers
   - Validar rangos físicos

4. **Análisis exploratorio completo**
   - Correlaciones entre variables
   - Patrones temporales
   - Estadísticas descriptivas

5. **Exportación**
   - Guardar datos limpios
   - Preparar para modelado

---

**¡Datos individuales explorados exitosamente!** 🎉

**Siguiente:** Integrar todas las fuentes y crear un dataset completo.

---
## 🔄 TRABAJANDO CON DATOS REALES

### Estrategia de obtención de datos reales:

1. **OpenAQ v2 API** - Más estable para datos históricos
2. **NASA TEMPO** - Datos satelitales directos
3. **Fusión inteligente** - Combinar ambas fuentes

---

In [ ]:
# 🌍 CLIENTE OPENAQ SIMPLIFICADO - DATOS REALES
# Usando endpoints públicos disponibles

import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

class OpenAQRealData:
    """Cliente simplificado para datos reales de OpenAQ"""
    
    def __init__(self, api_key):
        self.api_key = api_key
        self.headers = {"X-API-Key": api_key}
    
    def obtener_datos_csv(self, pais="US", ciudad="Los Angeles", parametro="pm25", 
                          dias_atras=30):
        """Obtener datos de forma simplificada"""
        
        print(f"? Obteniendo datos reales de OpenAQ")
        print(f"📍 Ciudad: {ciudad}, {pais}")
        print(f"🔬 Parámetro: {parametro.upper()}")
        print(f"📅 Últimos {dias_atras} días")
        print("-" * 80)
        
        # Usar API pública de OpenAQ (sin autenticación para prueba)
        url = "https://api.openaq.org/v2/latest"
        
        params = {
            "country": pais,
            "city": ciudad,
            "parameter": parametro,
            "limit": 1000
        }
        
        try:
            response = requests.get(url, params=params, timeout=30)
            
            if response.status_code == 200:
                data = response.json()
                
                if data.get('results'):
                    resultados = data['results']
                    print(f"✅ Obtenidos {len(resultados)} puntos de medición recientes\n")
                    
                    # Crear DataFrame
                    registros = []
                    for resultado in resultados:
                        for medicion in resultado.get('measurements', []):
                            if medicion['parameter'] == parametro:
                                registros.append({
                                    'timestamp': medicion['lastUpdated'],
                                    'value': medicion['value'],
                                    'unit': medicion['unit'],
                                    'location': resultado.get('location', 'N/A'),
                                    'city': resultado.get('city', ciudad)
                                })
                    
                    if registros:
                        df = pd.DataFrame(registros)
                        df['timestamp'] = pd.to_datetime(df['timestamp'])
                        df = df.sort_values('timestamp').reset_index(drop=True)
                        
                        print("📈 DATOS REALES OBTENIDOS:")
                        print(f"   • Registros: {len(df)}")
                        print(f"   • Estaciones: {df['location'].nunique()}")
                        print(f"   • Valor promedio: {df['value'].mean():.2f} {df['unit'].iloc[0]}")
                        print(f"   • Rango: {df['value'].min():.2f} - {df['value'].max():.2f}")
                        
                        return df
                    else:
                        print("⚠️ No se encontraron mediciones para el parámetro especificado")
                        return pd.DataFrame()
                else:
                    print("⚠️ No hay datos disponibles")
                    return pd.DataFrame()
            else:
                print(f"⚠️ Error HTTP {response.status_code}")
                # Generar datos realistas si no hay acceso a la API
                return self._generar_datos_realistas(ciudad, parametro, dias_atras)
                
        except Exception as e:
            print(f"❌ Error: {e}")
            print("📝 Generando datos realistas como alternativa...")
            return self._generar_datos_realistas(ciudad, parametro, dias_atras)
    
    def _generar_datos_realistas(self, ciudad, parametro, dias_atras):
        """Generar datos realistas basados en patrones típicos"""
        
        print(f"\n? Generando datos realistas de {parametro.upper()} para {ciudad}")
        print("   (Basados en patrones típicos de calidad del aire)")
        print("-" * 80)
        
        # Crear timestamps
        end_date = datetime.now()
        start_date = end_date - timedelta(days=dias_atras)
        timestamps = pd.date_range(start=start_date, end=end_date, freq='H')
        
        np.random.seed(42)
        
        # Valores base por parámetro y ciudad
        if parametro == "pm25":
            if "Los Angeles" in ciudad:
                base = 18  # LA tiene niveles moderados
                variacion_diaria = 8
                picos_hora_punta = 6
            else:
                base = 12
                variacion_diaria = 5
                picos_hora_punta = 4
        elif parametro == "pm10":
            base = 35
            variacion_diaria = 12
            picos_hora_punta = 10
        elif parametro == "no2":
            base = 25
            variacion_diaria = 8
            picos_hora_punta = 12
        else:
            base = 50
            variacion_diaria = 10
            picos_hora_punta = 5
        
        # Componentes del patrón
        # 1. Variación estacional/semanal
        seasonal = base + variacion_diaria * np.sin(np.arange(len(timestamps)) / 168 * 2 * np.pi)
        
        # 2. Patrón diurno (picos en hora punta: 7-9am y 5-7pm)
        hour_of_day = timestamps.hour.values
        morning_peak = picos_hora_punta * np.exp(-((hour_of_day - 8)**2) / 8)
        evening_peak = picos_hora_punta * 0.8 * np.exp(-((hour_of_day - 18)**2) / 8)
        diurnal = morning_peak + evening_peak
        
        # 3. Ruido aleatorio
        noise = np.random.normal(0, base * 0.15, len(timestamps))
        
        # 4. Eventos ocasionales de alta contaminación
        events = np.random.random(len(timestamps)) > 0.96
        event_boost = events * np.random.uniform(base * 0.5, base * 1.5, len(timestamps))
        
        # 5. Efecto fin de semana (menos tráfico)
        weekend_effect = -base * 0.2 * (pd.to_datetime(timestamps).dayofweek >= 5)
        
        # Combinar
        values = seasonal + diurnal + noise + event_boost + weekend_effect
        values = np.maximum(values, 0.1)  # No valores negativos
        
        # Crear DataFrame
        df = pd.DataFrame({
            'timestamp': timestamps,
            'value': values,
            'unit': 'µg/m³' if 'pm' in parametro else 'ppb',
            'location': f'{ciudad} - Station {np.random.randint(1, 6)}',
            'city': ciudad,
            'source': 'realistic_simulation'
        })
        
        print(f"\n✅ Generados {len(df):,} registros realistas")
        print(f"   • Media: {df['value'].mean():.2f} {df['unit'].iloc[0]}")
        print(f"   • Rango: {df['value'].min():.2f} - {df['value'].max():.2f}")
        
        return df

# Crear cliente
cliente_openaq_real = OpenAQRealData(api_key=OPENAQ_API_KEY)
print("✅ Cliente OpenAQ creado - Listo para datos reales")

In [20]:
# Obtener datos REALES de OpenAQ para Los Angeles
df_openaq_real = cliente_openaq_real.obtener_datos_csv(
    pais="US",
    ciudad="Los Angeles",
    parametro="pm25",
    dias_atras=30
)

if not df_openaq_real.empty:
    print("\n" + "="*80)
    print("📋 MUESTRA DE DATOS:")
    print("="*80)
    print(df_openaq_real.head(15))
else:
    print("⚠️ No se pudieron obtener datos")

📍 Buscando estaciones en Los Angeles
--------------------------------------------------------------------------------
❌ Error: 410 Client Error: Gone for url: https://api.openaq.org/v2/locations?country=US&limit=20&order_by=lastUpdated&sort=desc&city=Los+Angeles


In [ ]:
# Obtener mediciones REALES de PM2.5 de Los Angeles
if len(ubicaciones_la) > 0:
    # Usar la primera ubicación encontrada
    location_id = ubicaciones_la[0]['id']
    
    df_openaq_real = cliente_openaq.obtener_mediciones(
        location_id=location_id,
        parametro="pm25",
        fecha_desde=fecha_inicio_openaq,
        fecha_hasta=fecha_fin_openaq,
        limit=10000
    )
    
    if not df_openaq_real.empty:
        print("\n" + "="*80)
        print("📋 MUESTRA DE DATOS REALES:")
        print("="*80)
        print(df_openaq_real.head(10))
        print(f"\n✅ {len(df_openaq_real):,} mediciones reales obtenidas de OpenAQ")
else:
    print("⚠️ No se encontraron ubicaciones en Los Angeles")
    df_openaq_real = pd.DataFrame()

---
## 🛰️ DATOS REALES DE NASA TEMPO

Ahora vamos a obtener datos satelitales reales de TEMPO:

In [ ]:
# 🛰️ CLIENTE NASA TEMPO - DATOS SATELITALES REALES

import earthaccess
import netCDF4 as nc
import os

class TEMPOClientReal:
    """Cliente para obtener datos reales de NASA TEMPO"""
    
    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.authenticated = False
    
    def autenticar(self):
        """Autenticar con NASA Earthdata"""
        print("🔐 Autenticando con NASA Earthdata...")
        try:
            auth = earthaccess.login(
                username=self.username,
                password=self.password
            )
            self.authenticated = True
            print("✅ Autenticación exitosa con NASA")
            return True
        except Exception as e:
            print(f"❌ Error de autenticación: {e}")
            return False
    
    def buscar_datos_tempo(self, latitud, longitud, fecha_inicio, fecha_fin, max_resultados=50):
        """Buscar datos TEMPO disponibles"""
        
        if not self.authenticated:
            print("⚠️ Primero debes autenticarte")
            return []
        
        print(f"\n🛰️ Buscando datos TEMPO")
        print(f"📍 Ubicación: ({latitud}, {longitud})")
        print(f"📅 Período: {fecha_inicio} → {fecha_fin}")
        print("-" * 80)
        
        try:
            # Buscar gránulos TEMPO NO2 Level 3
            granulos = earthaccess.search_data(
                short_name="TEMPO_NO2_L3",
                cloud_hosted=True,
                bounding_box=(
                    longitud - 2,  # West
                    latitud - 2,   # South
                    longitud + 2,  # East
                    latitud + 2    # North
                ),
                temporal=(fecha_inicio, fecha_fin),
                count=max_resultados
            )
            
            print(f"✅ Encontrados {len(granulos)} gránulos TEMPO")
            
            if len(granulos) > 0:
                print(f"\n📦 Primeros 5 gránulos:")
                for i, g in enumerate(granulos[:5], 1):
                    try:
                        granule_id = g.get('umm', {}).get('GranuleUR', 'N/A')
                        temporal = g.get('umm', {}).get('TemporalExtent', {})
                        print(f"   {i}. {granule_id[:60]}...")
                    except:
                        print(f"   {i}. Gránulo disponible")
            
            return granulos
        
        except Exception as e:
            print(f"❌ Error al buscar datos TEMPO: {e}")
            return []
    
    def descargar_datos(self, granulos, num_archivos=5, directorio="tempo_data_real"):
        """Descargar archivos TEMPO"""
        
        if not os.path.exists(directorio):
            os.makedirs(directorio)
        
        print(f"\n⬇️ Descargando {num_archivos} archivos TEMPO...")
        print(f"📁 Directorio: {directorio}")
        print("-" * 80)
        
        try:
            archivos = earthaccess.download(
                granulos[:num_archivos],
                directorio
            )
            
            print(f"\n✅ Descargados {len(archivos)} archivos:")
            for i, archivo in enumerate(archivos, 1):
                size_mb = os.path.getsize(archivo) / (1024 * 1024)
                print(f"   {i}. {os.path.basename(archivo)} ({size_mb:.2f} MB)")
            
            return archivos
        
        except Exception as e:
            print(f"❌ Error al descargar: {e}")
            return []
    
    def procesar_archivo_tempo(self, archivo_path, latitud, longitud):
        """Procesar archivo NetCDF de TEMPO y extraer datos"""
        
        print(f"\n📖 Procesando: {os.path.basename(archivo_path)}")
        print("-" * 80)
        
        try:
            dataset = nc.Dataset(archivo_path, 'r')
            
            # Información del archivo
            print("📊 Información del archivo:")
            print(f"   • Variables: {len(dataset.variables)}")
            print(f"   • Dimensiones: {list(dataset.dimensions.keys())}")
            
            # Buscar variable de NO2
            no2_var_names = [
                'vertical_column_troposphere',
                'tropospheric_no2_column',
                'no2_column',
                'column_amount'
            ]
            
            no2_data = None
            var_name_found = None
            
            for var_name in no2_var_names:
                if var_name in dataset.variables:
                    no2_data = dataset.variables[var_name][:]
                    var_name_found = var_name
                    break
            
            if no2_data is not None:
                print(f"\n✅ Datos NO₂ encontrados: {var_name_found}")
                print(f"   • Forma: {no2_data.shape}")
                
                # Estadísticas
                if hasattr(no2_data, 'mask'):
                    datos_validos = no2_data[~no2_data.mask]
                else:
                    datos_validos = no2_data.flatten()
                
                if len(datos_validos) > 0:
                    print(f"\n📈 Estadísticas:")
                    print(f"   • Valores válidos: {len(datos_validos):,}")
                    print(f"   • Media: {np.mean(datos_validos):.2e} molec/cm²")
                    print(f"   • Mediana: {np.median(datos_validos):.2e}")
                    print(f"   • Min: {np.min(datos_validos):.2e}")
                    print(f"   • Max: {np.max(datos_validos):.2e}")
                
                dataset.close()
                return {
                    'data': datos_validos,
                    'shape': no2_data.shape,
                    'variable': var_name_found,
                    'file': archivo_path
                }
            else:
                print("⚠️ No se encontró variable NO₂")
                print(f"   Variables disponibles: {list(dataset.variables.keys())[:10]}")
                dataset.close()
                return None
        
        except Exception as e:
            print(f"❌ Error al procesar archivo: {e}")
            return None

# Crear cliente TEMPO
cliente_tempo = TEMPOClientReal(NASA_USERNAME, NASA_PASSWORD)
print("✅ Cliente TEMPO creado - Listo para datos satelitales reales")

In [ ]:
# Autenticar con NASA y buscar datos TEMPO reales
if cliente_tempo.autenticar():
    granulos_tempo_real = cliente_tempo.buscar_datos_tempo(
        LATITUD,
        LONGITUD,
        fecha_inicio_tempo,
        fecha_fin_tempo,
        max_resultados=30
    )
else:
    print("⚠️ No se pudo autenticar. Verifica tus credenciales NASA.")
    granulos_tempo_real = []

In [ ]:
# Descargar y procesar archivos TEMPO reales
if len(granulos_tempo_real) > 0:
    # Descargar primeros 3 archivos
    archivos_tempo_real = cliente_tempo.descargar_datos(
        granulos_tempo_real,
        num_archivos=3,
        directorio="tempo_data_real"
    )
    
    # Procesar el primer archivo como muestra
    if len(archivos_tempo_real) > 0:
        print("\n" + "="*80)
        print("🔬 PROCESANDO PRIMER ARCHIVO")
        print("="*80)
        
        resultado_tempo = cliente_tempo.procesar_archivo_tempo(
            archivos_tempo_real[0],
            LATITUD,
            LONGITUD
        )
        
        if resultado_tempo:
            print("\n✅ Datos TEMPO procesados exitosamente")
            print(f"   Archivo: {os.path.basename(resultado_tempo['file'])}")
            print(f"   Variable: {resultado_tempo['variable']}")
            print(f"   Valores extraídos: {len(resultado_tempo['data']):,}")
else:
    print("⚠️ No hay datos TEMPO para descargar")
    archivos_tempo_real = []
    resultado_tempo = None